# Modelo matematico

In [13]:
# Modulos
import numpy as np
import matplotlib.pyplot as plt
import sympy as spy
import control as ctl
from sympy.core.compatibility import iterable
from reliability.Other_functions import crosshairs as csr

%matplotlib qt

# Funciones auxiliares
def mem2mem(equation,opp):
    """mem2mem:
        Aplica una operacion sencilla en ambos miembros de una ecuacion.

    Args:
        equation: Ecuacion a operar.
        opp: Termino para operar en ambos miembros.

    Returns: 
        Ecuacion de resultado luego de operar.
    """
    sel=opp[0]
    expr=spy.sympify(opp[1::])
    
    if sel=="+":
        return spy.Eq(equation.lhs+expr,equation.rhs+expr)
    elif sel=="-":
        return spy.Eq(equation.lhs-expr,equation.rhs-expr)
    elif sel=="*":
        return spy.Eq(equation.lhs*expr,equation.rhs*expr)
    elif sel=="/":
        return spy.Eq(equation.lhs/expr,equation.rhs/expr)

def disp(ecuaciones,titulo=""):
    """disp:
        Mostrar una ecuacion con titulo y separadores

    Args:
        ecuaciones: Ecuacion para imprimir
        titulo: Titulo opcional a mostrar. Defaults to "".
    """
    print(titulo + "\n")
    
    if type(ecuaciones)!=iterable:
        spy.pprint(ecuaciones,num_columns=200)
    
    else:
        for ecc in ecuaciones:
            spy.pprint(ecc,num_columns=200)

    print("\n" + "-"*80)


# Idea del sistema

**Disco dentado que gira persiguiendo el rayo de luz.**

![](img/sistema.jpg)

In [2]:
#simbolos (en este orden)
# tension/corriente del motor, resistencia/inductancia de bobinados, cte de fcem, velocidad del rotor
V,Im,rm,lm,kv,Wm=spy.symbols("V,Im,rm,lm,kv,Wm")
# torque motor, momento de inercia/friccion del rotor
Tm,Jm,s,bm,T1=spy.symbols("Tm,Jm,s,bm,T1")
# torque/velocidad del 2do eje, momento de inercia/friccion del segidor
T2,W2,J,b2=spy.symbols("T2,W2,J,b2")

# cte del motor, relacion de reduccion 
km,n=spy.symbols("km,n")

#igualdades auxiliares
# Tm = km*Im
# n = N1/N2 = W2/W1 = T1/T2

#ecuaciones
# V = Im*(rm+lm*s) + kv*Wm
# Tm = Wm*(Jm*s + bm) + T1
# T2 = W2*(J*s + b2)
eqq1=spy.Eq(V,Im*(rm+lm*s) + kv*Wm)
eqq2=spy.Eq(Tm,Wm*(Jm*s + bm) + T1)
eqq3=spy.Eq(T2,W2*(J*s + b2))

disp((eqq1,eqq2,eqq3),"Ecuaciones:")

#cancelar T1 y T1
aux0=spy.Eq(T1,spy.solve(eqq2,T1)[0])
aux1=mem2mem(eqq3,"*n").replace(T2*n,T1)

aux2=spy.Eq(aux0.lhs-aux1.lhs,aux0.rhs-aux1.rhs)

disp((eqq1,aux2),"Cancelar T1 y T2:")

#cancelar Tm e Im
aux3=spy.Eq(Tm,spy.solve(aux2,Tm)[0])
aux4=spy.Eq(Im,spy.solve(eqq1,Im)[0])
aux4=mem2mem(aux4,"*km").replace(Im*km,Tm)

aux5=spy.Eq(aux3.lhs-aux4.lhs,aux3.rhs-aux4.rhs)

disp(aux5,"Cancelar Im y Tm:")

#resolver para W2/V
aux6=spy.Eq(Wm,spy.solve(aux5,Wm)[0])
aux6=mem2mem(aux6,"*n").replace(Wm*n,W2)

aux7=spy.Eq(W2,spy.solve(aux6,W2)[0])
aux7=mem2mem(aux7,"/V")

aux7=spy.Eq(W2/V,aux7.rhs.collect(s))

disp(aux7,"Resolver para W2/V:")

#armar funcion de transferencia G = P2/V
aux8=mem2mem(aux7,"/s")

P2=spy.symbols("P2")
eqqG=spy.Eq(P2/V,aux8.rhs)

disp(eqqG,"Funcion de transferencia:")


Ecuaciones:

(V = Im⋅(lm⋅s + rm) + Wm⋅kv, Tm = T₁ + Wm⋅(Jm⋅s + bm), T₂ = W₂⋅(J⋅s + b₂))

--------------------------------------------------------------------------------
Cancelar T1 y T2:

(V = Im⋅(lm⋅s + rm) + Wm⋅kv, 0 = -Jm⋅Wm⋅s + Tm - W₂⋅n⋅(J⋅s + b₂) - Wm⋅bm)

--------------------------------------------------------------------------------
Cancelar Im y Tm:

                                           km⋅(V - Wm⋅kv)
0 = J⋅W₂⋅n⋅s + Jm⋅Wm⋅s + W₂⋅b₂⋅n + Wm⋅bm - ──────────────
                                             lm⋅s + rm   

--------------------------------------------------------------------------------
Resolver para W2/V:

W₂                                             km⋅n                                          
── = ────────────────────────────────────────────────────────────────────────────────────────
V        2                       2 ⎛      2        ⎞     ⎛   2                     2        ⎞
     b₂⋅n ⋅rm + bm⋅rm + km⋅kv + s ⋅⎝J⋅lm⋅n  + Jm⋅lm⎠ + s⋅⎝J⋅n ⋅rm + Jm⋅rm + b

# Funcion de transferencia a lazo abierto

* Se mide desde una referencia el angulo del disco y el angulo de incidencia.
* Se corrije con el motor hasta que el error sea cero.

![](img/bloques.jpg)

In [3]:
#armar funcion de transferencia de lazo abierto
# cte para el sensor de LDRs, cte para el pwm, ganancia para alimentar el motor
ssr,pwm,Av,TFOL=spy.symbols("ssr,pwm,Av,TFOL")

eqqTFOL=spy.Eq(TFOL,ssr*pwm*Av*eqqG.rhs)

disp(eqqTFOL,"Funcion de transferencia a lazo abierto:")

Funcion de transferencia a lazo abierto:

                                             Av⋅km⋅n⋅pwm⋅ssr                                       
TFOL = ────────────────────────────────────────────────────────────────────────────────────────────
         ⎛    2                       2 ⎛      2        ⎞     ⎛   2                     2        ⎞⎞
       s⋅⎝b₂⋅n ⋅rm + bm⋅rm + km⋅kv + s ⋅⎝J⋅lm⋅n  + Jm⋅lm⎠ + s⋅⎝J⋅n ⋅rm + Jm⋅rm + b₂⋅lm⋅n  + bm⋅lm⎠⎠

--------------------------------------------------------------------------------


# Planta final

Motor:

![](img/motor.jpg)

In [6]:
#parametros de la planta
#sensor(V/°): +/-5V a +/- 20°
ppssr=5/20
#pwm(%D/V): 5V es 100% Duty
pppwm=1/5 
#ganancia(V/%D): 100% Duty son 24V
ppAv=24/100

eqqTFOL=eqqTFOL.subs({ssr:ppssr, 
                      pwm:pppwm,
                      Av:ppAv})

#motor
pprm = 26.9 
pplm = 600E-6
ppJm = 1.5E-7
ppkm = 30.82E-3
ppkv = 30.82E-3

ppbm = 3.7628E-5

eqqTFOL=eqqTFOL.subs({rm:pprm, 
                      lm:pplm,
                      Jm:ppJm,
                      km:ppkm,
                      kv:ppkv,
                      bm:ppbm})

#caja reductora
ppn=1/4

#seguidor
#disco dentado de pla: h=1cm, r=5cm, densidad=1.32gr/cm3, J=1/2*densidad*vol*r**2
densidad=1.32/1000
radio=5
vol=(np.pi*radio**2)*1

#inercia (gr*cm2)
ppJ=1/2*densidad*vol*radio**2
#pasar a Kg*m2
ppJ=ppJ*1E-7
#friccion 
ppb2=2*ppbm

eqqTFOL=eqqTFOL.subs({n:ppn, 
                      J:ppJ,
                      b2:ppb2}).expand()

disp(eqqTFOL,"Planta:")

num=[spy.numer(eqqTFOL.rhs)]
den=[spy.denom(eqqTFOL.rhs).coeff(s,pot) for pot in reversed(range(4))]

print("Coeficientes:")
print((num,den))

Planta:

                                    9.246e-5                             
TFOL = ──────────────────────────────────────────────────────────────────
                             3                        2                  
       9.48596511360217e-11⋅s  + 4.27827325926497e-6⋅s  + 0.00208858975⋅s

--------------------------------------------------------------------------------
Coeficientes:
([9.24600000000000e-5], [9.48596511360217e-11, 4.27827325926497e-6, 0.00208858975000000, 0])


# Funcion de transferencia a lazo abierto

In [8]:
# crear funcion de transferencia
TFOL=ctl.tf([9.24600000000000e-5], [9.48596511360217e-11, 4.27827325926497e-6, 0.00208858975000000, 0])

print("Transferencia a lazo abierto:")
print(TFOL)

Transferencia a lazo abierto:

                9.246e-05
------------------------------------------
9.486e-11 s^3 + 4.278e-06 s^2 + 0.002089 s



# Funcion de transferencia a lazo cerrado

In [9]:
#realimentacion unitaria
H=1
TFCL=ctl.feedback(TFOL,H)

print("Transferencia a lazo cerrado:")
print(TFCL)

Transferencia a lazo cerrado:

                      9.246e-05
------------------------------------------------------
9.486e-11 s^3 + 4.278e-06 s^2 + 0.002089 s + 9.246e-05



# Respuesta al escalon original

In [14]:
#respuesta al escalon
t,y=ctl.step_response(TFCL,1000)

fig=plt.figure()
plt.plot(t,y)
plt.grid()
csr()
plt.draw()

# while plt.waitforbuttonpress()!=True:
#     pass
plt.close(fig)